In [32]:
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.remote.webelement import WebElement
from selenium import webdriver
from selenium.webdriver.remote import webelement
from selenium.common.exceptions import StaleElementReferenceException, ElementClickInterceptedException
from IPython.display import Image, display
import os
import sys
from time import sleep

import pandas as pd
import gspread
from google.auth import default
from google.auth.exceptions import RefreshError

In [33]:
from selenium import webdriver
import time

options = webdriver.ChromeOptions()
options.add_experimental_option("detach", True)
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--disable-extensions")
options.add_argument("--no-sandbox")
options.add_argument("--disable-infobars")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-browser-side-navigation")
options.add_argument("--disable-gpu")

driver = webdriver.Chrome(options=options)
driver.get("https://www.google.com")
# driver.execute_script("window.open('https://nowsecure.nl/', '_blank')")
# time.sleep(15)
# driver.switch_to.window(driver.window_handles[1])

# driver.switch_to.frame(0)

# driver.find_element(By.XPATH, '//*[@id="challenge-stage"]/div/label/input').click()


In [30]:
def login_indeed_google(driver, google_email, google_password):
    """
    Function to log in to Indeed via Google login.
    
    Parameters:
    driver (webdriver): The Selenium WebDriver instance.
    google_email (str): The Google email address for the Indeed account.
    google_password (str): The Google account password.
    """
    driver.get("https://secure.indeed.com/account/login")
    
    try:
        # Wait for the Google login button and click it
        google_login_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "login-google-button"))
        )
        google_login_button.click()

        # Switch to Google login window
        main_window = driver.current_window_handle
        WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))
        for handle in driver.window_handles:
            if handle != main_window:
                driver.switch_to.window(handle)
                break

        # Enter email and submit
        email_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "identifierId"))
        )
        email_input.send_keys(google_email)
        email_input.send_keys(Keys.RETURN)
        time.sleep(2)

        # Enter password and submit
        password_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, "Passwd"))
        )
        password_input.send_keys(google_password)
        password_input.send_keys(Keys.RETURN)

        # Wait for Google authentication to complete
        WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(1))
        driver.switch_to.window(main_window)

        # Click "Continue" button if it exists
        try:
            continue_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "button.VfPpkd-LgbsSe.VfPpkd-LgbsSe-OWXEXe-INsAgc"))
            )
            continue_button.click()
        except TimeoutException:
            print("Nút 'Continue' không tồn tại hoặc không xuất hiện.")

        # Verify login success on Indeed
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "user-account-link"))
        )
        print("Logged in successfully via Google!")

    except TimeoutException as e:
        print(f"Error: Timeout occurred - {e}")
    except NoSuchElementException as e:
        print(f"Error: Element not found - {e}")
    except Exception as e:
        print(f"Error during login via Google: {e}")


In [34]:
username = "chidinhm3@gmail.com"
password = "dinh0911"

# Ensure the function login_indeed_google is defined before calling it
login_indeed_google(driver, username, password)


Nút 'Continue' không tồn tại hoặc không xuất hiện.
Error: Timeout occurred - Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7624E6CC5+28821]
	(No symbol) [0x00007FF762453850]
	(No symbol) [0x00007FF7622F578A]
	(No symbol) [0x00007FF7623491BE]
	(No symbol) [0x00007FF7623494AC]
	(No symbol) [0x00007FF762392647]
	(No symbol) [0x00007FF76236F33F]
	(No symbol) [0x00007FF76238F412]
	(No symbol) [0x00007FF76236F0A3]
	(No symbol) [0x00007FF76233A778]
	(No symbol) [0x00007FF76233B8E1]
	GetHandleVerifier [0x00007FF76281FCCD+3408029]
	GetHandleVerifier [0x00007FF76283743F+3504143]
	GetHandleVerifier [0x00007FF76282B61D+3455469]
	GetHandleVerifier [0x00007FF7625ABDCB+835995]
	(No symbol) [0x00007FF76245EB6F]
	(No symbol) [0x00007FF76245A824]
	(No symbol) [0x00007FF76245A9BD]
	(No symbol) [0x00007FF76244A1A9]
	BaseThreadInitThunk [0x00007FFD766CE8D7+23]
	RtlUserThreadStart [0x00007FFD7781FBCC+44]



In [35]:
def scrape_job_data(driver, link):
    all_jobs_data = []
    seen_jobs = set()  # Sử dụng set để lưu các công việc đã xử lý

    # Mở trang đầu tiên
    driver.get(link)

    for page in range(1, 6):
        print(f"Đang xử lý trang {page}")

        # Tìm tất cả các job card trên trang
        job_cards = driver.find_elements(By.XPATH, "//tr/td[@class='resultContent css-lf1alc eu4oa1w0']")

        # Kiểm tra số lượng job card tìm được
        print(f"Đã tìm thấy {len(job_cards)} công việc trên trang {page}.")

        # Duyệt qua từng job card và lấy thông tin
        for index, job in enumerate(job_cards, start=1):
            try:
                # Tên công ty
                try:
                    company_name = job.find_element(By.XPATH, ".//span[@data-testid='company-name']").text
                except NoSuchElementException:
                    company_name = 'N/A'

                # Địa điểm
                try:
                    location = job.find_element(By.XPATH, ".//div[@data-testid='text-location']").text
                except NoSuchElementException:
                    location = 'N/A'

                # Tiêu đề công việc
                try:
                    job_title = job.find_element(By.XPATH, ".//h2[@class='jobTitle css-1psdjh5 eu4oa1w0']//span").text
                except NoSuchElementException:
                    job_title = 'N/A'

                # Mức lương
                try:
                    salary = job.find_element(By.XPATH, ".//div[@class='metadata salary-snippet-container css-1f4kgma eu4oa1w0']//div[@data-testid='attribute_snippet_testid']").text
                except NoSuchElementException:
                    salary = 'N/A'

                # Liên kết công việc
                try:
                    link_job = job.find_element(By.XPATH, ".//h2[@class='jobTitle css-1psdjh5 eu4oa1w0']//a").get_attribute('href')
                except NoSuchElementException:
                    link_job = 'N/A'

                job_data = (link_job, company_name, job_title, salary, location)

                # Kiểm tra nếu công việc đã tồn tại trong set
                if job_data not in seen_jobs:
                    print(f"Công việc {index} trên trang {page}:")
                    print(f"  Tên công ty: {company_name}")
                    print(f"  Vị trí: {location}")
                    print(f"  Tiêu đề công việc: {job_title}")
                    print(f"  Mức lương: {salary}")
                    print(f"  Liên kết: {link_job}")
                    print("-" * 50)

                    # Lưu thông tin công việc
                    all_jobs_data.append([link_job, company_name, job_title, salary, location])
                    seen_jobs.add(job_data)

            except Exception as e:
                print(f"Lỗi khi xử lý công việc {index} trên trang {page}: {e}")

        # Chuyển đến trang tiếp theo nếu không phải trang cuối
        if page < 5:
            try:
                next_button = driver.find_element(By.XPATH, f"//a[@data-testid='pagination-page-{page + 1}']")
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
                next_button.click()
                time.sleep(2)  # Đợi trang tải

            except NoSuchElementException:
                print("Không tìm thấy nút chuyển trang. Dừng tại trang hiện tại.")
                break

    return all_jobs_data


In [36]:
import gspread
from google.oauth2.service_account import Credentials
try:
    # Đường dẫn đến tệp JSON của bạn
    SERVICE_ACCOUNT_FILE = 'snappy-charge-444008-u3-2d010cb44a06.json'

    # Phạm vi quyền truy cập
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

    # Xác thực với Google Sheets API
    credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    gc = gspread.authorize(credentials)

    # Mở Google Sheet bằng URL
    spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1FUAXm33xCAOQtlJgvn7h6jqCXDPYH9AuFBpVOOB3AnY/edit?usp=sharing'
    sheet = gc.open_by_url(spreadsheet_url)

    # Lấy trang tính đầu tiên
    work_sheet = sheet.get_worksheet(0)

    # Lấy hàng tiêu đề
    header = work_sheet.row_values(1)

    # Kiểm tra sự tồn tại của các tiêu đề cần thiết
    required_columns = ["Company name", "Title job", "Location job", "Salary"]
    for column in required_columns:
        if column not in header:
            print(f"Column '{column}' not found in the header.")
            exit(1)

    # Tìm vị trí các cột theo tiêu đề
    company_col = header.index("Company name") + 1  # Cột "Company name"
    job_col = header.index("Title job") + 1         # Cột "Title job"
    locate_col = header.index("Location job") + 1  # Cột "Location job"
    salary_col = header.index("Salary") + 1        # Cột "Salary"

    # Kiểm tra kết nối bằng cách in tiêu đề
    print(f"Successfully accessed sheet: {sheet.title}")
    print("Headers:", header)

    # Đọc tất cả dữ liệu
    data = work_sheet.get_all_records()
    print("Dữ liệu từ Google Sheet:", data)

except Exception as e:
    print(f"Error: {str(e)}")


Successfully accessed sheet: Crawling info job Indeed
Headers: ['Link job', 'Company name', 'Title job', 'Salary', 'Location job']
Dữ liệu từ Google Sheet: [{'Link job': 'https://ca.indeed.com/jobs?q=CFA&l=Vancouver%2C+BC&vjk=2f3201c73baa7f64', 'Company name': 'Doane Grant Thornton', 'Title job': 'Senior Associate, Restructuring', 'Salary': '$65,000–$89,000 a year', 'Location job': 'Vancouver, BC\n&nbsp;\nHybrid work', '': ''}, {'Link job': 'https://ca.indeed.com/jobs?q=CFA&l=Vancouver%2C+BC&vjk=2f3201c73baa7f64', 'Company name': 'Doane Grant Thornton', 'Title job': 'Senior Associate, Restructuring', 'Salary': '$65,000–$89,000 a year', 'Location job': 'Vancouver, BC\n&nbsp;\nHybrid work', '': ''}, {'Link job': 'https://ca.indeed.com/jobs?q=CFA&l=Vancouver%2C+BC&vjk=fee7029be196d7c3', 'Company name': 'EDC', 'Title job': 'Regional Director, Mid', 'Salary': '$20 an hour', 'Location job': 'Vancouver, BC\n&nbsp;\nHybrid work', '': ''}, {'Link job': 'https://ca.indeed.com/jobs?q=CFA&l=Vancou

In [10]:
import gspread
from google.oauth2.service_account import Credentials

def update_google_sheet(sheet, data):
    """
    Cập nhật Google Sheet với dữ liệu công việc đã thu thập.

    Args:
        sheet: Đối tượng Google Sheet.
        data: Danh sách các dữ liệu công việc để thêm.
    """
    try:
        # Kiểm tra xem dữ liệu có rỗng không
        if not data:
            print("Không có dữ liệu để cập nhật.")
            return

        # Chuẩn bị dữ liệu dưới dạng danh sách các danh sách
        # Mỗi danh sách con tương ứng với một hàng dữ liệu
        # Giả sử data là danh sách các danh sách hoặc danh sách các từ điển
        # Nếu data là danh sách các từ điển, chuyển đổi nó thành danh sách các danh sách
        if isinstance(data[0], dict):
            # Xác định thứ tự cột theo tiêu đề đã lấy trước đó
            headers = sheet.row_values(1)
            prepared_data = []
            for row in data:
                prepared_row = [
                    row.get("Link job", ""),
                    row.get("Company name", ""),
                    row.get("Title job", ""),
                    row.get("Salary", ""),
                    row.get("Location job", "")
                ]
                prepared_data.append(prepared_row)
        else:
            # Nếu data đã là danh sách các danh sách
            prepared_data = data

        # Sử dụng append_rows để thêm nhiều hàng cùng một lúc
        sheet.append_rows(prepared_data, value_input_option='RAW')
        print(f"Đã thêm thành công {len(prepared_data)} hàng vào Google Sheet.")

    except Exception as e:
        print(f"Lỗi khi cập nhật Google Sheet: {str(e)}")


In [37]:
link = "https://vn.indeed.com/jobs?q=lao+%C4%91%E1%BB%99ng+ph%E1%BB%95+th%C3%B4ng&l=Th%C3%A0nh+ph%E1%BB%91+H%E1%BB%93+Ch%C3%AD+Minh&from=searchOnHP%2Cwhatautocomplete&vjk=fa79e1eb3dcf33b7"
jobs = scrape_job_data(driver, link)


Đang xử lý trang 1
Đã tìm thấy 15 công việc trên trang 1.
Công việc 1 trên trang 1:
  Tên công ty: Nội Thất Nem
  Vị trí: Thành phố Hồ Chí Minh
  Tiêu đề công việc: THỢ MỘC/THỢ PHỤ/LAO ĐỘNG PHỔ THÔNG
  Mức lương: 9,000,000 VNĐ - 20,500,000 VNĐ a month
  Liên kết: https://vn.indeed.com/rc/clk?jk=4731e149cc5ad87a&bb=qlSTAafZxifR1HbWEhR_kjV95by9Rtrvv7670Xl6B2qkyEiXo7m2EN9Xnbap7sCXcoeqvTSygG3IdHWdWFfFN-Ab9dqPa4kRTQhifsF4Fw4QhKC0qLGBz04oGnFMnCTc&xkcb=SoBz67M3496Ffty5dB0LbzkdCdPP&fccid=6d30e53491e590d2&vjs=3
--------------------------------------------------
Công việc 2 trên trang 1:
  Tên công ty: Công Ty TNHH Sản Xuất Thương Mại Hồng Nam
  Vị trí: Thành phố Hồ Chí Minh
  Tiêu đề công việc: Lao Động Phổ Thông (Đóng Gói Giao Hàng)
  Mức lương: 7,000,000 VNĐ - 10,000,000 VNĐ a month
  Liên kết: https://vn.indeed.com/rc/clk?jk=4114b5466b6145c4&bb=qlSTAafZxifR1HbWEhR_krmAeXcwYGETAuAxzXjC6Qv7lSoEc4LfG8ApeYkwcFwpid0cYf_liyTxogmcVhfMy-yqrG4vOIksrRQeZnGtvR8dI_HoLM9vTiQjpTTD_eFG&xkcb=SoDH67M3496Ffty

In [8]:
print("Dữ liệu công việc thu thập:"+str(jobs))

Dữ liệu công việc thu thập:[['https://vn.indeed.com/rc/clk?jk=4731e149cc5ad87a&bb=7MPg9D0EkgcYbQ5AWJHLylCrI3FhxeulCbnsb8yCWPwaQR6ZSJ6XjP12f9twVaZ-0u1db3arxLGLd6RI86AictkeHIlIe7gnIdt9qjBsi70TLLbaZ7vbi-4MSLXTSclS&xkcb=SoDS67M349m1VmgcvL0LbzkdCdPP&fccid=6d30e53491e590d2&vjs=3', 'Nội Thất Nem', 'THỢ MỘC/THỢ PHỤ/LAO ĐỘNG PHỔ THÔNG', '9,000,000 VNĐ - 20,500,000 VNĐ a month', 'Thành phố Hồ Chí Minh'], ['https://vn.indeed.com/rc/clk?jk=4114b5466b6145c4&bb=7MPg9D0EkgcYbQ5AWJHLyg6BIvV5e7iFn6aj7Y6JBZ41nkUFbRpWMVsjMUY_1ofVRjvSRv_0n3jQ8-MKOtPREQd3um--dvVpi7JMJkJ5qVfvi4d4NLfWF4BNWEtq6kBi&xkcb=SoBm67M349m1VmgcvL0KbzkdCdPP&fccid=dd616958bd9ddc12&vjs=3', 'Nội Thất Nem', 'THỢ MỘC/THỢ PHỤ/LAO ĐỘNG PHỔ THÔNG', '9,000,000 VNĐ - 20,500,000 VNĐ a month', 'Thành phố Hồ Chí Minh'], ['https://vn.indeed.com/rc/clk?jk=5b4574fa5916a8ee&bb=7MPg9D0EkgcYbQ5AWJHLyndRmsTof_VTzEbdt9BNZKVQyTIyWVQ45j-SCm1SBRHYG0qFPGaRHuM3cW8ITgYlh6cDzDh3SC7LZiaO82xeTN1ZMmwqK8IdN3KKWpiQHLEY&xkcb=SoD767M349m1VmgcvL0JbzkdCdPP&fccid=d4ca8c57f

In [23]:
update_google_sheet(work_sheet, jobs)

Đã thêm thành công 75 hàng vào Google Sheet.
